In [1]:
import pandas as pd
import numpy as np
# import os
import nltk
# nltk.data.path.append(r"C:\Users\liufa\nltk_data")

# from nltk.tokenize import word_tokenize
# nltk.download('brown')
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


# BERT
import tensorflow as tf
import keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Lambda,\
    GlobalMaxPooling1D, LSTM, Bidirectional, concatenate, Embedding, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification, AdamW
from transformers import TFBertModel, TFDistilBertModel, TFBertForSequenceClassification

import gc
gc.enable()

In [2]:
raw_data=pd.read_csv(r'C:\Users\liufa\Desktop\Kaggle competition\train.csv')

df = raw_data.rename(columns={'excerpt':'text','target':'y'})
df.head(5)

,id,url_legal,license,text,y,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [3]:
import string
string.punctuation

def remove_punct(text):
    text_nopunct=''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['text_no_punct']=df['text'].apply(lambda x: remove_punct(x))

import re
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['text_tokenized']=df['text_no_punct'].apply(lambda x: tokenize(x.lower()))

# import nltk
# stopwords=nltk.corpus.stopwords.words('english')
# def remove_stopwords(tokenized_list):
#     text=[word for word in tokenized_list if word not in stopwords]
#     return text
# df['text_no_stopwords']=df['text_tokenized'].apply(lambda x: remove_stopwords(x))

# wn=nltk.WordNetLemmatizer()
# def lemmatizing(tokenized_text):
#     text=[wn.lemmatize(word) for word in tokenized_text]
#     return text
# df['text_lemmatizer']=df['text_no_stopwords'].apply(lambda x: lemmatizing(x))

# df['text_clean']=df['text_lemmatizer'].apply(lambda x: ' '.join(x))
# df.head()

In [4]:
df['sentence_len']=df['text'].apply(lambda x: len(x)-x.count(' '))
df['word_len']=df['text_tokenized'].apply(lambda x: len(x))
def count_punct(text):
    ct=sum([1 for char in text if char in string.punctuation])
    return round(ct/(len(text)-text.count(' ')),3)
df['punct_count']=df['text'].apply(lambda x:count_punct(x))
df.head()

,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,sentence_len,word_len,punct_count
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...",819,179,0.033
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...",774,169,0.072
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...",747,166,0.063
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...",747,164,0.044
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...",577,147,0.055


In [5]:
df.columns

Index(['id', 'url_legal', 'license', 'text', 'y', 'standard_error',
       'text_no_punct', 'text_tokenized', 'sentence_len', 'word_len',
       'punct_count'],
      dtype='object')

In [6]:
# help functions

# to define 'rmse' as loss instead of 'mse'
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def softMaxAxis1(x):
    return tf.keras.activations.softmax(x,axis=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor = 0.5 , 
    patience=1, 
    mode='min', 
    verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    verbose=1,
    restore_best_weights=True)

# checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     "model_fold_{}".format(fold) +".h5", 
#     monitor='val_loss', 
#     save_best_only=True, 
#     mode='min', 
#     verbose=1)

In [7]:
# BERT tokenizer
# 'bert-base-cased', 'distilbert-base-cased', 'roberta-base', 'distilroberta-base'
Model_path = 'roberta-base'
Max_len = 300

def bert_tokenize(X, model_path=Model_path, max_len=Max_len):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenized = tokenizer(X.tolist(), padding='max_length',truncation=True, max_length=max_len, return_tensors="np") 
    return {feat: tokenized[feat] for feat in tokenizer.model_input_names}

def build_bert_model(model_path=Model_path, max_len=Max_len):

    input_ids = tf.keras.Input(shape=(max_len, ),dtype='int32')
    attention_mask = tf.keras.Input(shape=(max_len, ),dtype='int32')

    bert_output = TFAutoModel.from_pretrained(model_path,
                                              output_hidden_states=True, 
                                              hidden_dropout_prob=0.0,
                                              layer_norm_eps=1e-7)(input_ids, attention_mask)

    last_hidden_states = bert_output.hidden_states[-1]
    # cls_token = bert_layer[:, 0, :]

    # weight layer
    weight_layer = Dense(512, activation='tanh')(last_hidden_states)
    weight_layer = Dense(1, activation=softMaxAxis1)(weight_layer)

    #  attention layer for weighted sum over the 300 token embedding vectors
    layer = multiply([weight_layer,last_hidden_states])
    layer = Lambda(lambda x: K.sum(x, axis=1))(layer)

    # regression layer
    out = Dense(1, activation='linear')(layer)
    model = tf.keras.Model(inputs=[input_ids , attention_mask], outputs=out)
    model.compile(optimizer=Adam(2e-5), loss = [rmse])
    return model

In [8]:
# K-fold CV training
kf = KFold(n_splits = 5 , shuffle = True , random_state = 36)
cv_predictions = np.zeros(len(df))

for fold , (train_index , val_index) in enumerate(kf.split(df)):
    print('-'*50)
    print("Training Fold {}".format(fold))

    X_train, y_train = df.iloc[train_index]['text'], df.iloc[train_index][['y']].values
    X_val, y_val = df.iloc[val_index]['text'], df.iloc[val_index][['y']].values
    
    X_train_tokenized = bert_tokenize(X_train)
    X_val_tokenized = bert_tokenize(X_val)
    
    model = build_bert_model(Model_path, Max_len)
    save_path = "model_fold_{}".format(fold) +".h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=save_path, 
                                                    monitor='val_loss', 
                                                    save_best_only=True, 
                                                    mode='min', 
                                                    verbose=1)
    
    history = model.fit([X_train_tokenized['input_ids'],X_train_tokenized['attention_mask']], y_train, 
                     epochs=15, batch_size=5, 
                     validation_data = ([X_val_tokenized['input_ids'],X_val_tokenized['attention_mask']], y_val), 
                     callbacks=[reduce_lr, early_stop],
                     verbose=1)

    # Predict validation set to save them in the out of folds array
    val_pred = model.predict([X_val_tokenized['input_ids'],X_val_tokenized['attention_mask']])
    cv_predictions[val_index] = val_pred.reshape(-1)
    
    del model
    gc.collect()
    K.clear_session()

# Calculate out of folds root mean squared error
cv_rmse = np.sqrt(mean_squared_error(df['y'], cv_predictions))
print(f'Our out of folds RMSE is {cv_rmse}')

--------------------------------------------------
Training Fold 0


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/15
454/454 [==============================] - 134s 273ms/step - loss: 0.6218 - val_loss: 0.5143
Epoch 2/15
454/4

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/15
454/454 [==============================] - 131s 269ms/step - loss: 0.6051 - val_loss: 0.5570
Epoch 2/15
454/454 [==============================] - 121s 266ms/step - loss: 0.4369 - val_loss: 0.5198
Epoch 3/15
454/454 [==============================] - 120s 265ms/step - loss: 0.3112 - val_loss: 0.4732
Epoch 4/15
454/454 [==============================] - 121s 265ms/step - loss: 0.2534 - val_loss: 0.4587
Epoch 5/15
454/454 [==============================] - 120s 265ms/step - loss: 0.2284 - val_loss: 0.4744

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 6/15
454/454 [==============================] - 120s 265ms/step - loss: 0.1716 - val_loss: 0.4612

Epoch 00006: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
--------------------------------------------------
Training Fold 2


...

Epoch 1/15
454/454 [==============================] - 132s 269ms/step - loss: 0.6274 - val_loss: 0.5483
Epoch 2/15
454/454 [==============================] - 121s 266ms/step - loss: 0.4336 - val_loss: 0.5076
Epoch 3/15
454/454 [==============================] - 121s 266ms/step - loss: 0.3220 - val_loss: 0.4945
Epoch 4/15
454/454 [==============================] - 121s 266ms/step - loss: 0.2565 - val_loss: 0.4871
Epoch 5/15
454/454 [==============================] - 121s 266ms/step - loss: 0.2335 - val_loss: 0.4860
Epoch 6/15
454/454 [==============================] - 121s 266ms/step - loss: 0.2159 - val_loss: 0.4909

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 7/15
454/454 [==============================] - 121s 266ms/step - loss: 0.1656 - val_loss: 0.4786
Epoch 8/15
454/454 [==============================] - 123s 272ms/step - loss: 0.1100 - val_loss: 0.4772
Epoch 9/15
454/454 [==============================] - 122s 269ms/step - loss: 0.0975 - 

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/15
454/454 [==============================] - 132s 271ms/step - loss: 0.6366 - val_loss: 0.5136
Epoch 2/15
454/454 [==============================] - 121s 267ms/step - loss: 0.4361 - val_loss: 0.5188

Epoch 00002: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 3/15
454/454 [==============================] - 121s 266ms/step - loss: 0.2963 - val_loss: 0.4777
Epoch 4/15
454/454 [==============================] - 122s 270ms/step - loss: 0.2052 - val_loss: 0.4769
Epoch 5/15
454/454 [==============================] - 121s 267ms/step - loss: 0.1703 - val_loss: 0.4680
Epoch 6/15
454/454 [==============================] - 121s 266ms/step - loss: 0.1456 - val_loss: 0.4833

Epoch 00006: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 7/15
454/454 [==============================] - 122s 269ms/step - loss: 0.1205 - val_loss: 0.4764

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Restoring model weights from 

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/15
454/454 [==============================] - 133s 272ms/step - loss: 0.6144 - val_loss: 0.5296
Epoch 2/15
454/454 [==============================] - 121s 267ms/step - loss: 0.4425 - val_loss: 0.5192
Epoch 3/15
454/454 [==============================] - 121s 267ms/step - loss: 0.3305 - val_loss: 0.4899
Epoch 4/15
454/454 [==============================] - 121s 267ms/step - loss: 0.2602 - val_loss: 0.4719
Epoch 5/15
454/454 [==============================] - 121s 267ms/step - loss: 0.2306 - val_loss: 0.4631
Epoch 6/15
454/454 [==============================] - 121s 267ms/step - loss: 0.2020 - val_loss: 0.4900

Epoch 00006: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 7/15
454/454 [==============================] - 121s 267ms/step - loss: 0.1565 - val_loss: 0.4535
Epoch 8/15
454/454 [==============================] - 121s 267ms/step - loss: 0.1098 - val_loss: 0.4517
Epoch 9/15
454/454 [==============================] - 121s 267ms/step - loss: 0.1033 - 

In [ ]:
del model
gc.collect()
K.clear_session()

In [8]:
# skipping k-fold training for quick model testing
# test & train split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df[['y']].values, test_size=0.2, random_state=42)

X_train_tokenized = bert_tokenize(X_train)
X_test_tokenized = bert_tokenize(X_test)

print (
str(len(X_train_tokenized))\
+ ' x ' +\
str(len(X_train_tokenized['input_ids']))\
+ ' x ' +\
str(len(X_train_tokenized['input_ids'][0]))
)

display(X_train_tokenized)

2 x 2267 x 300


{'input_ids': array([[    0,   133,   745, ...,     1,     1,     1],
        [    0,   133,  1114, ...,     1,     1,     1],
        [    0, 37818,     5, ...,     1,     1,     1],
        ...,
        [    0,   100,  1017, ...,     1,     1,     1],
        [    0,  1121,   209, ...,     1,     1,     1],
        [    0, 39488,    16, ...,     1,     1,     1]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [9]:
model = build_bert_model(Model_path, Max_len)

history = model.fit([X_train_tokenized['input_ids'],\
                     X_train_tokenized['attention_mask']],\
                     y_train, epochs=15, batch_size=5, validation_split=0.35, 
                     callbacks=[reduce_lr,early_stop],
                     verbose=1)

score = model.evaluate([X_test_tokenized['input_ids'],\
                        X_test_tokenized['attention_mask']],\
                        y_test, 
                        verbose=1)
print("Test Scores:", score)

K.clear_session()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/15
295/295 [==============================] - 98s 300ms/step - loss: 0.6511 - val_loss: 0.5751
Epoch 2/15
295/29

In [ ]:
# debugging

intermediate_layer_model =tf.keras.Model(inputs=model.input,
                                 outputs=model.get_layer('tf_roberta_model').output)
intermediate_output = intermediate_layer_model.predict([X_test_tokenized['input_ids'][0:10],\
                        X_test_tokenized['attention_mask'][0:10]])

In [ ]:
intermediate_output[0][0]

In [ ]:
intermediate_output[0]

In [ ]:
intermediate_output[0].shape

In [ ]:
# plotting training/validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
############################### Testing & Validation ################################
BERTmodel = TFBertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
a = tokenizer(df['text'].tolist()[0:5],padding=True,return_tensors='tf')

In [ ]:
BERTmodel(a).last_hidden_state

In [ ]:
BERTmodel(a)[0]

In [ ]:
print(len(BERTmodel(a).last_hidden_state))
print(len(BERTmodel(a).last_hidden_state[0]))
print(len(BERTmodel(a).last_hidden_state[0][0]))